# Exploring & Cleaning Neighbourhoods in Toronto, Canada

In [1]:
# import required modules
import requests
import bs4 
import lxml.html as lh
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

In [2]:
# intended website
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
# Create a handle, page to handle the contents of the intended website
page = requests.get(url)

In [4]:
# Store the contents of the website under a doc
doc = lh.fromstring(page.content)

In [5]:
# Parse data that are stored between <tr>..<tr> of HTML
tr_elements = doc.xpath('//tr')

In [6]:
# Check the length of the first 15 rows for sanity check. This means that all rows have 3 columns which implies that operation is successful.
[len(T) for T in tr_elements[:15]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [7]:
# Parsing first row as header
tr_elements = doc.xpath('//tr')

# Create an empty list
col=[]
i=0

# Storing each element (header)
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [8]:
# Creating Pandas DataFrame
for j in range(1, len(tr_elements)):
    T=tr_elements[j]
    
    # if row is not of size 3, the //tr data is not from the table we wanted
    if len(T)!=3:
        break
        
    # i is the index of the column
    i=0
    
    # Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content().replace("\n", "")
        # check if row is empty
        if i>0:
        # convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        # append the data to the empty list of the 1th column
        col[i][1].append(data)
        # increment i for the next column
        i+=1

In [9]:
# Sanity check : this shows each of the 3 columns has exactly 289 values
[len(C) for (title,C) in col]

[289, 289, 289]

In [10]:
# Create DataFrame
Dict={title:column for (title, column) in col}
df=pd.DataFrame(Dict)

In [11]:
df.shape

(289, 3)

In [12]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [13]:
df.tail()

,Postcode,Borough,Neighbourhood
284,M8Z,Etobicoke,Mimico NW
285,M8Z,Etobicoke,The Queensway West
286,M8Z,Etobicoke,Royal York South West
287,M8Z,Etobicoke,South of Bloor
288,M9Z,Not assigned,Not assigned


In [14]:
# Making a copy of df to make changes so as to retain original, in case required.
df1 = df.copy()

In [15]:
# Cleaning up column headers
df1.columns = df1.columns.str.strip()

In [16]:
# Checking column headers
df1.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [17]:
# Sanity check to see if df1 is same as df
df1.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [18]:
# Drop rows where Borough is 'Not assigned'
df1 = df1[df1.Borough != 'Not assigned']

In [19]:
# Check shape of df1 to ensure that rows where Borough is Not assigned are dropped
df1.shape

(212, 3)

In [20]:
# Check head of df1 to verify changes have been made
df1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [21]:
# Comibining Neighbourhood with common postal code & Resetting Index
df1 = pd.DataFrame(df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join) )
df1 = df1.reset_index()

In [22]:
# Checking the new df1 with combined neighbourhood for common postcode
df1.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [23]:
# For those not assigned neighbourhood, they will assume that of the borough
df1.Neighbourhood[df1.Neighbourhood == 'Not assigned'] = df1.Borough

In [24]:
# Checking the new df1 after accounting for the not assigned neighbourhood to be that of the borough 
# and to combine neighbourhoods for common postcode
df1.shape

(103, 3)

# Obtaining the Latititude and Longtitude Coordinates 

In [25]:
# Reading the geospatial data csv file
geospatial_data = pd.read_csv('Geospatial_Coordinates.csv')

In [26]:
# Checking column headers
geospatial_data.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [27]:
# Making a DataFrame
geospatial_data = pd.DataFrame(geospatial_data)

In [28]:
# Checking first 5 lines of DataFrame
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
# Joining the 2 DataFrames
canada_loc = pd.concat([df1, geospatial_data], axis=1, join='inner')

In [30]:
# Checking first 5 lines of combined DataFrame
canada_loc.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [31]:
# Dropping duolicate column 
canada_loc.drop(['Postal Code'], axis=1, inplace=True)

In [37]:
# Checking the 1st 5 lines of combined dataframe
canada_loc.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [32]:
# The 2 DataFrames can also be combined using Join instead of concat as above
canada_loc1 = df1.join(geospatial_data)

In [33]:
# Checking 1st 5 lines of joined DataFrame
canada_loc1.head()

,Postcode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [34]:
# Dropping duolicate column 
canada_loc1.drop(['Postal Code'], axis=1, inplace=True)

In [35]:
# Checking shape of dropped DataFrame (103 rows and 3 columns)
canada_loc1.shape

(103, 5)